In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /kaggle/working/
%cd /content/

/content


In [ ]:
!pip install gdown
!gdown 1x77fyA9BONMHnQXhvxP-87hFTCFwGm9a -O hw1_data.zip

# Unzip the downloaded zip file
!unzip ./hw1_data.zip


流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: hw1_data/p1_data/mini/train/19540.jpg  
  inflating: hw1_data/p1_data/mini/train/36374.jpg  
  inflating: hw1_data/p1_data/mini/train/34074.jpg  
  inflating: hw1_data/p1_data/mini/train/31551.jpg  
  inflating: hw1_data/p1_data/mini/train/23462.jpg  
  inflating: hw1_data/p1_data/mini/train/7131.jpg  
  inflating: hw1_data/p1_data/mini/train/29410.jpg  
  inflating: hw1_data/p1_data/mini/train/2489.jpg  
  inflating: hw1_data/p1_data/mini/train/24586.jpg  
  inflating: hw1_data/p1_data/mini/train/28243.jpg  
  inflating: hw1_data/p1_data/mini/train/32326.jpg  
  inflating: hw1_data/p1_data/mini/train/11668.jpg  
  inflating: hw1_data/p1_data/mini/train/19491.jpg  
  inflating: hw1_data/p1_data/mini/train/13556.jpg  
  inflating: hw1_data/p1_data/mini/train/7751.jpg  
  inflating: hw1_data/p1_data/mini/train/1.jpg  
  inflating: hw1_data/p1_data/mini/train/28806.jpg  
  inflating: hw1_data/p1_data/mini/train/5624.jpg  
  inflating: hw1_data/p1_da

In [ ]:
%cd hw1_data

/content/hw1_data


In [ ]:
import os

import torch
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms

In [ ]:
!pip install torchsummary

## P1

### P1 pretrained resnet50 backbone model with miniImageNet

In [ ]:
!pip install byol-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 14.7 MB/s eta 0:00:00a 0:00:01


In [ ]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomRotation(
            degrees=20
        ),
    transforms.RandomHorizontalFlip(p=0.1),
    transforms.RandomVerticalFlip(p=0.1),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])

In [ ]:
class ImgDataset(Dataset):
    # data loading
    def __init__(self, root_dir):
        self.cache = {}
        self.x = []
        for i in os.listdir(root_dir):
          self.x.append(os.path.join(root_dir, i))
        # self.y = []
    def read_image(self, path):
      x = torchvision.io.read_image(path).float()
      x = TRANSFORM_IMG(x)

      return x

    # working for indexing
    def __getitem__(self, index):
        if self.x[index] not in self.cache:
            self.cache[self.x[index]] = self.read_image(self.x[index])

        return self.cache[self.x[index]]

    # return the length of our dataset
    def __len__(self):
        return len(self.x)

In [ ]:
images_loader = DataLoader(ImgDataset(os.path.join("p1_data", "mini", "train")), batch_size=128, shuffle=True)

In [ ]:
from torchsummary import summary
from torch import nn

resnet = torchvision.models.resnet50(pretrained=False)
num_features = resnet.fc.in_features

resnet.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 64)
)
summary(resnet.cuda(), input_size=(3,128,128))

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]           4,096
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
           Conv2d-11          [-1, 256, 32, 32]          16,384
      BatchNorm2d-12          [-1, 256, 32, 32]             512
           Conv2d-13          [-1, 256, 32, 32]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
from byol_pytorch import BYOL

learner = BYOL(
    resnet,
    image_size = 128,
    hidden_layer = 'avgpool'
)

opt = torch.optim.Adam(learner.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=2)

In [ ]:
from tqdm import tqdm

epochs = 200
step = 0

learner = learner.to('cuda')

run_parallel = False

if run_parallel:
  learner = nn.parallel.DataParallel(learner)

for epoch in range(epochs):
  epoch += 1
  pbar = tqdm(images_loader)
  pbar.set_description(f"Training epoch [{epoch}/{epochs}]")
  total_loss = 0

  for i, images in enumerate(pbar):
    i += 1
    step += 1
    images = images.cuda()
    outputs_loss = learner(images)
    opt.zero_grad()

    if run_parallel:
      losses = outputs_loss
      loss = losses.mean()
    else:
      loss = outputs_loss

    loss.backward()
    opt.step()
    learner.update_moving_average() # update moving average of target encoder
    total_loss += loss.item()
    pbar.set_postfix(loss = f"{total_loss / i:.4f}")

    if step % 2000 == 0:
      torch.save(resnet.state_dict(), f'./pretrained_{epoch}_{i}_{step}.pt')

torch.save(resnet.state_dict(), f'./pretrained_{epoch}.pt')
#   scheduler.step(total_loss / i)

Training epoch [21/200]:   7%|▋         | 21/300 [00:16<02:46,  1.67it/s, loss=0.4551]

### P1 finetune resnet50 backbone + classifier

In [ ]:
import pandas as pd

class ImgDataset(Dataset):
    # data loading
    def __init__(self, root_dir, data_type, num_classes):
        assert data_type in ("train", "val")

        self.num_classes = num_classes
        self.x = []
        self.cache = {}
        self.file_map_label = dict()

        label_path = os.path.join(root_dir, f"{data_type}.csv")
        data = pd.read_csv(label_path)

        root_dir = os.path.join(root_dir, data_type)
        for i in os.listdir(root_dir):
          self.x.append(os.path.join(root_dir, i))

        for x, y in zip(data['filename'].tolist(), data['label'].tolist()):
            self.file_map_label[os.path.join(root_dir, x)] = int(y)



    def read_image(self, path):
        x = torchvision.io.read_image(path).float()

        if x.shape[1] != x.shape[2]:
          max_bound = max(x.shape[1:])

          new_x = torch.zeros((3, max_bound, max_bound))
          new_x[:, :x.shape[1], :x.shape[2]] = x
          x = new_x

        x = transforms.Resize((128, 128))(x)

        return x

    def make_label(self, label):
        y = torch.zeros(self.num_classes)
        y[label] = 1

        return y

    # working for indexing
    def __getitem__(self, index):
        if self.x[index] not in self.cache:
            self.cache[self.x[index]] = self.read_image(self.x[index])

        return self.cache[self.x[index]], self.file_map_label[self.x[index]]

    # return the length of our dataset
    def __len__(self):
        return len(self.x)

In [ ]:
num_classes = 65
batch_size=64

In [ ]:
train_loader = DataLoader(ImgDataset(os.path.join("p1_data", "office"), "train", num_classes), batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(ImgDataset(os.path.join("p1_data", "office"), "val", num_classes), batch_size=batch_size, shuffle=False)

In [ ]:
from tqdm import tqdm
from torch import nn
import math
import random
import copy

def criterion(a, f):
    return nn.CrossEntropyLoss()(a, f)

def evaluate(model, valid_loader):
    model.eval()
    correct = 0
    total = 0

    pbar = tqdm(valid_loader)
    pbar.set_description(f"Evaluating")

    for (x, y) in pbar:
        x = x.cuda()
        x = transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))(x)
        y = y.cuda()
        outputs = model(x)
        pred_label = torch.argmax(outputs, dim=1)
        label = y

        for (a, f) in zip(pred_label, label):
            if a == f:
                correct += 1
            total += 1

        accuracy = correct / total
        pbar.set_postfix(acc = f"{accuracy:.4f}")

    model.train()
    return accuracy

def train(model, epochs=50, lr=0.01, model_prefix="", run_parallel=False, scheduler_unfreeze=False, TRANSFORM_IMG=None):
    history = { "accuracy": float(0), "model": None }

    opt = torch.optim.Adam(model.parameters(), lr=lr)

    step = 0

    model = model.to('cuda')

    if scheduler_unfreeze:
        for params in model.parameters():
            params.requires_grad = False

        for params in model.fc.parameters():
            params.requires_grad = True

    if run_parallel:
      model = nn.parallel.DataParallel(model)

    model_layers = len(list(model.parameters()))

    for epoch in range(epochs):
      epoch += 1

      if scheduler_unfreeze and epoch % 5 == 0:
          layer_index = 0
          for params in model.parameters():
              layer_index += 1

              if model_layers - layer_index <= math.ceil(model_layers / epochs) * epoch:
                  params.requires_grad = True

      if scheduler_unfreeze and epoch >= epochs // 10 * 9:
        for params in model.parameters():
            params.requires_grad = True

      pbar = tqdm(train_loader)
      pbar.set_description(f"Training epoch [{epoch}/{epochs}]")
      total_loss = 0

      for i, (x, y) in enumerate(pbar):
        i += 1
        step += 1

        if TRANSFORM_IMG is not None:
          target_transform = random.randint(0, len(TRANSFORM_IMG) - 1)
          x = TRANSFORM_IMG[target_transform](x)

        x = x.cuda()
        y = y.cuda()
        outputs = model(x)
        outputs_loss = criterion(outputs, y)

        opt.zero_grad()

        if run_parallel:
          losses = outputs_loss
          loss = losses.mean()
        else:
          loss = outputs_loss

        loss.backward()
        opt.step()

        total_loss += loss.item()
        pbar.set_postfix(loss = f"{total_loss / i:.4f}")

        # if step % 2000 == 0:
        #   torch.save(resnet.state_dict(), f'./{model_prefix}_{epoch}_{i}_{step}.pt')

      accuracy = evaluate(model, valid_loader)
      if accuracy >= history['accuracy']:
          history['accuracy'] = accuracy
          history['model'] = copy.deepcopy(model)
      # else:
      #     model = copy.deepcopy(history['model'])
    torch.save(resnet.state_dict(), f'./{model_prefix}.pt')
#     return model


### Setting A

In [ ]:
resnet = torchvision.models.resnet50(pretrained=False)
num_features = resnet.fc.in_features

resnet.fc = nn.Sequential(
    nn.Linear(num_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, num_classes)
)

In [ ]:
train(resnet, 200, "setting_a", True)

### Setting B

In [ ]:
resnet = torchvision.models.resnet50(pretrained=False)
num_features = resnet.fc.in_features

resnet.load_state_dict(torch.load("./p1_data/pretrain_model_SL.pt"))

resnet.fc = nn.Sequential(
    nn.Linear(num_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, num_classes)
)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-48-7f7b331d5322>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

In [ ]:
TRANSFORM_IMG = [
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.Resize((128, 128))
    ]),
]

train(resnet, 20, 0.001, "setting_b_transfer_learning", False, scheduler_unfreeze=True, TRANSFORM_IMG=TRANSFORM_IMG)

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 15.96it/s, acc=0.2069]


In [ ]:
TRANSFORM_IMG = [
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.GaussianBlur(kernel_size=3),          # 添加高斯模糊
      transforms.Resize((128, 128))

    ]),
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.Resize((128, 128))
    ]),
]

train(resnet, 20, 0.001, "setting_b_transfer_learning1", False, scheduler_unfreeze=True, TRANSFORM_IMG=TRANSFORM_IMG)

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 16.03it/s, acc=0.1970]


In [ ]:
TRANSFORM_IMG = [
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.RandomVerticalFlip(p=0.5),    # 隨機垂直翻轉
      transforms.RandomHorizontalFlip(p=0.5),  # 隨機水平翻轉
      # transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
      # transforms.RandomRotation(degrees=20),
      # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),  # 隨機仿射變換
      # transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # 隨機透視變形
      # transforms.RandomInvert(p=0.5),                  # 隨機顛倒顏色
      # transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
      transforms.Resize((128, 128))
    ]),
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.GaussianBlur(kernel_size=3),          # 添加高斯模糊
      transforms.Resize((128, 128))

    ]),
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.Resize((128, 128))
    ]),
]

train(resnet, 20, 0.001, "setting_b_transfer_learning2", False, scheduler_unfreeze=True, TRANSFORM_IMG=TRANSFORM_IMG)

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 15.94it/s, acc=0.2217]


In [ ]:
TRANSFORM_IMG = [
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.RandomVerticalFlip(p=0.5),    # 隨機垂直翻轉
      transforms.RandomHorizontalFlip(p=0.5),  # 隨機水平翻轉
      # transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
      transforms.RandomRotation(degrees=20),
      transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),  # 隨機仿射變換
      # transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # 隨機透視變形
      # transforms.RandomInvert(p=0.5),                  # 隨機顛倒顏色
      # transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
      transforms.Resize((128, 128))
    ]),
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.GaussianBlur(kernel_size=3),          # 添加高斯模糊
      transforms.Resize((128, 128))

    ]),
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.Resize((128, 128))
    ]),
]

train(resnet, 20, 0.001, "setting_b_transfer_learning3", False, scheduler_unfreeze=True, TRANSFORM_IMG=TRANSFORM_IMG)

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 15.92it/s, acc=0.2980]


In [ ]:
TRANSFORM_IMG = [
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.RandomVerticalFlip(p=0.5),    # 隨機垂直翻轉
      transforms.RandomHorizontalFlip(p=0.5),  # 隨機水平翻轉
      # transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
      transforms.RandomRotation(degrees=20),
      transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),  # 隨機仿射變換
      # transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # 隨機透視變形
      transforms.RandomInvert(p=0.5),                  # 隨機顛倒顏色
      transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
      transforms.Resize((128, 128))
    ]),
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.GaussianBlur(kernel_size=3),          # 添加高斯模糊
      transforms.Resize((128, 128))

    ]),
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.Resize((128, 128))
    ]),
]

train(resnet, 20, 0.001, "setting_b_transfer_learning4", False, scheduler_unfreeze=True, TRANSFORM_IMG=TRANSFORM_IMG)

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 16.01it/s, acc=0.3153]


In [ ]:
TRANSFORM_IMG = [
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.RandomVerticalFlip(p=0.5),    # 隨機垂直翻轉
      transforms.RandomHorizontalFlip(p=0.5),  # 隨機水平翻轉
      # transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
      transforms.RandomRotation(degrees=20),
      transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),  # 隨機仿射變換
      transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # 隨機透視變形
      transforms.RandomInvert(p=0.5),                  # 隨機顛倒顏色
      transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
      transforms.Resize((128, 128))
    ]),
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.GaussianBlur(kernel_size=3),          # 添加高斯模糊
      transforms.Resize((128, 128))

    ]),
    transforms.Compose([
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
      transforms.Resize((128, 128))
    ]),
]

train(resnet, 20, 0.001, "setting_b_transfer_learning5", False, scheduler_unfreeze=True, TRANSFORM_IMG=TRANSFORM_IMG)

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 16.10it/s, acc=0.3030]


In [ ]:
train(resnet, 100, 0.0001, "setting_b_transfer_learning6", False, scheduler_unfreeze=False, TRANSFORM_IMG=TRANSFORM_IMG)

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 15.97it/s, acc=0.4064]


In [ ]:
resnet.load_state_dict(torch.load("./setting_b_transfer_learning6.pt"))

<ipython-input-94-a11a351bccd0>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet.load_state_dict(torch.load("./setting_b_transfer_learning6.pt"))


<All keys matched successfully>

In [ ]:
train(resnet, 100, 0.00005, "setting_b_transfer_learning7", False, scheduler_unfreeze=False, TRANSFORM_IMG=TRANSFORM_IMG)

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 15.76it/s, acc=0.4261]


In [ ]:
from google.colab import files
files.download('setting_b_transfer_learning7.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Setting C

In [ ]:
resnet = torchvision.models.resnet50(pretrained=False)
num_features = resnet.fc.in_features

resnet.fc = nn.Sequential(
    nn.Linear(num_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, num_classes)
)

# resnet.load_state_dict(torch.load("./pretrained_50.pt"))
resnet.load_state_dict(torch.load("./pretrained_47_200_14000.pt"))
num_features = resnet.fc[0].in_features

/tmp/ipykernel_36/1760555819.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet.load_state_dict(torch.load("./pretrained_47_200_14000.pt"))


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "fc.6.weight", "fc.6.bias". 
	Unexpected key(s) in state_dict: "fc.5.weight", "fc.5.bias". 
	size mismatch for fc.0.weight: copying a param with shape torch.Size([512, 2048]) from checkpoint, the shape in current model is torch.Size([1024, 2048]).
	size mismatch for fc.0.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for fc.3.weight: copying a param with shape torch.Size([128, 512]) from checkpoint, the shape in current model is torch.Size([512, 1024]).
	size mismatch for fc.3.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([512]).

In [ ]:
resnet.fc = nn.Sequential(
    nn.Linear(num_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, num_classes)
)

In [ ]:
for params in resnet.parameters():
    params.requires_grad = False

for params in resnet.fc.parameters():
    params.requires_grad = True

resnet = train(resnet, 100, 0.01, "setting_c", True,True)

Training epoch [55/100]:  50%|█████     | 4/8 [00:02<00:02,  1.89it/s, loss=4.0093]

In [ ]:
for params in resnet.parameters():
    params.requires_grad = True

train(resnet, 100, 0.001, "setting_c", True)

### Setting D

In [ ]:
resnet = torchvision.models.resnet50(pretrained=False)
num_features = resnet.fc.in_features

resnet.load_state_dict(torch.load("./p1_data/pretrain_model_SL.pt"))

resnet.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes)
)

for params in resnet.parameters():
    params.requires_grad = False

for params in resnet.fc.parameters():
    params.requires_grad = True

In [ ]:
train(resnet, 200, "setting_d", True)

### Setting E

In [ ]:
resnet = torchvision.models.resnet50(pretrained=False)
num_features = resnet.fc.in_features

resnet.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 64)
)

# resnet.load_state_dict(torch.load("./pretrained_50.pt"))
resnet.load_state_dict(torch.load("./pretrained_47_200_14000.pt"))
num_features = resnet.fc[0].in_features

In [ ]:
resnet.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes)
)


for params in resnet.parameters():
    params.requires_grad = False

for params in resnet.fc.parameters():
    params.requires_grad = True

In [ ]:
train(resnet, 200, "setting_e", True)

## P2

In [ ]:
class ImgDataset(Dataset):
    # data loading
    def __init__(self, root_dir):
        x = []
        for i in os.listdir(root_dir):
          if not i.endswith("mask.jpg"):
            x.append(os.path.join(root_dir), i)
        self.x = x

    def read_image(self, path):
      x = torchvision.io.read_image(path)
      y = torchvision.io.read_image(path.replace("sat.jpg", "mask.jpg"))

      return x, y

    # working for indexing
    def __getitem__(self, index):
        return self.read_image(self.x[index]) #, self.y[index]

    # return the length of our dataset
    def __len__(self):
        return len(self.x)